# SETUP

In [63]:
from selenium import webdriver
from bs4 import BeautifulSoup
from IPython.display import clear_output

import requests
import pandas as pd
import time

---

### brunch login

In [ ]:
url = "https://accounts.kakao.com/login?continue=https://kauth.kakao.com/oauth/authorize?client_id%3De0201caea90cafbb237e250f63a519b5%26response_type%3Dcode%26redirect_uri%3Dhttps%253A%252F%252Fbrunch.co.kr%252Fcallback%252Fauth%252Fkakao%26scope%3D%26state%3DaHR0cHM6Ly9icnVuY2guY28ua3IvL3NpZ25pbi9maW5pc2g_dXJsPSUyRg%26grant_type%3Dauthorization_code"
browser = webdriver.Chrome()
browser.get(url)

In [ ]:
brunch_id = browser.find_element_by_xpath('//*[@id="loginEmail"]')
brunch_id.send_keys('syh602@gmail.com')

pw = input('Enter Password : ')

brunch_pw = browser.find_element_by_xpath('//*[@id="loginPw"]')
brunch_pw.send_keys(pw)

brunch_submit = browser.find_element_by_class_name('submit')
brunch_submit.click()

In [17]:
browser.quit()

---

### comment crawling (Google Play App)

In [34]:
# open webdriver, get url
url = "https://play.google.com/store/apps/details?id=com.enuma.todomath&showAllReviews=true"
driver = webdriver.Chrome()
driver.get(url)

# make html, soup

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    print(last_height)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # wait 1 sec
    time.sleep(1)

    # calculate the length
    new_height = driver.execute_script('return document.body.scrollHeight')
    print(new_height, "\n")
    
    # move to 2400
    # driver.execute_script('window.scrollTo(0, 2400);')

    # break if scroll doesn't move
    if new_height == last_height:
        print("do not move any more, click /see more/ ")
        
        try:
            submit = driver.find_element_by_class_name('RveJvd')
            submit.click()
        except:
            print("There is no more pages, stop scrolling.")
            break

    last_height = new_height

6982
12364 

12364
17855 

17855
23063 

23063
23140 

23140
27842 

27842
27842 

do not move any more, click /see more/ 
27842
31414 

31414
31894 

31894
31894 

do not move any more, click /see more/ 
There is no more pages, stop scrolling.


In [ ]:
# get page sourse, parse html
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [49]:
# make list
date_list = []
score_list = []
text_list = []

# selecting
div_add = soup.find('div', {'class': 'LXrl4c'})
div_add_revise = div_add.findAll('span', {'jsname':"bN97Pc"})

# appending
for text in div_add_revise:
    text_list.append(text.get_text())

for k in range(len(text_list)):

    try:
        date = div_add.findAll('span',{'class':'p2TkOb'})[k].text
        date_list.append(date)

        if str(div_add.findAll('div',{'class':'pf5lIe'})[k].div['aria-label'])[11:12]==".":
            score = float(str(div_add.findAll('div',{'class':'pf5lIe'})[k].div['aria-label'])[10:13])
        else:
            score = float(str(div_add.findAll('div',{'class':'pf5lIe'})[k].div['aria-label'])[10:11])
        score_list.append(score)
    except:
        print("Error")
        continue

# check
len(date_list), len(text_list), len(score_list)

(245, 245, 245)

In [54]:
data_raw = pd.DataFrame(columns=['date', 'score', 'text'])
data_raw['date'] = date_list[:-1]
data_raw['score'] = score_list[1:]
data_raw['text'] = text_list[:-1]
data_raw.tail()

,date,score,text
239,2016년 6월 10일,5.0,굿
240,2016년 6월 14일,5.0,좋네요
241,2017년 5월 27일,5.0,좋아
242,2016년 6월 11일,5.0,굿 굿 굿
243,2016년 10월 28일,5.0,최고예요😄☺🙂😊😁


In [55]:
data_raw.to_pickle('todomath_data_03.pkl')

In [60]:
data_revise = data_raw.sort_values(by='date', ascending=False)
data_revise.reset_index(drop=True, inplace=True)
data_revise.tail()

,date,score,text
239,2016년 10월 23일,1.0,뭐....아이들 수학 잘되네요
240,2016년 10월 23일,5.0,저 4학년인데 왤케 쉬워요 1학년도 할수 있을것 같아요 아 진짜... 그래도 전 수...
241,2016년 10월 21일,5.0,수학을 힘들어하는 초등학교 자녀가 재미있어 하네요
242,2016년 10월 15일,1.0,다 좋은데 '사용 여부 문의'도 없이 자동결제는 좀. . .그렇네요. 여기와서 구독...
243,2016년 10월 10일,5.0,아들이 토도수학 좋아해요


In [61]:
data_revise.to_pickle('todomath_data_04.pkl')

In [62]:
driver.quit()

---

### stock crawling 

In [78]:
url = "https://finance.naver.com/sise/sise_deal_rank.nhn"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

# 인기검색어 리스트 가져오기
table_doc = soup.find('table', {'class': 'type_r1'})

# make list (name, stock)
name_list = []
stock_list = []

# append name, stock
for k in range(7):
    name_list.append(table_doc.find_all('a', {'class':'company'})[k].text)
    stock_list.append(table_doc.find_all('td', {'class':'number'})[k].text)

# make df (name, stock)
data_raw = pd.DataFrame(columns=['name', 'stock'])
data_raw['name'] = name_list
data_raw['stock'] = stock_list
data_raw

,name,stock
0,한화케미칼,"21,250"
1,KODEX 레버리지,"11,695"
2,KODEX 코스닥150선물인버스,"7,400"
3,삼성에스디에스,"213,500"
4,삼성전자우,"32,900"
5,LG전자,"66,000"
6,삼성바이오로직스,"404,000"
